<a href="https://colab.research.google.com/github/raihanalmenata/ML/blob/main/food_classifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roadmap
1. Convolution Layer
2. Max Pooling
3. 1 Hidden Layer Consist of 100 Neurons ReLu
4. Output Layer 2 Neurons, Softmax activiton function



## Import Required Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Image Data Generator

In [ ]:
def generators(TRAIN_DIR, VAL_DIR):

  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
  
  val_datagen = ImageDataGenerator(rescale=1./255)

  ##rotation range until fill_mode values were still placeholder

  train_generator = train_datagen.flow_from_directory(directory=TRAIN_DIR,
                                                      batch_size=100,
                                                      class_mode='categorical',
                                                      target_size=(128, 128))
  
  val_generator = val_datagen.flow_from_directory(directory=VAL_DIR,
                                                  batch_size=100,
                                                  class_mode='categorical',
                                                  target_size=(128, 128))
  
  return train_generator, val_generator

In [ ]:
# Test the generators
train_generator, val_generator = generators(TRAIN_DIR, VAL_DIR)

# Food Classification Model

Below is the code written to build the baseline version of the food classification model. 
The model is desgined to fulfill the minimum criteria of the model by implementing minimum configuration. The model then expected to represent the possible accuracy of the model with the estimated available dataset.

In [ ]:
def model():
  # The input is assumed to be 150x150 RGB Image
  model = tf.keras.models.Sequential(
      [tf.keras.layers.Conv2D(24, 7, activation='relu', input_shape=(128,128,3)), 
       tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
       tf.keras.layers.Conv2D(48, 5, activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)),
       tf.keras.layers.Conv2D(72, 3, activation='relu'),
       tf.keras.layers.MaxPool2D(pool_size=(2,2)),
       tf.keras.layers.Dropout(0.25),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Dense(10, activation='softmax')]) # The output label is configured to be 10

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

  return model

## Train Model

In [ ]:
model = model()
history = model.fit(train_data, epochs=100, verbose=1, validation_data=validation_data)

## Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


epochs = range(len(acc))


plt.plot(epochs, acc, 'r', 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', 'Validation Accuracy')
plt.title('Accuracy Comparison')
plt.show()
print('')


plt.plot(epochs, loss, 'r', 'Training Loss')
plt.plot(epochs, val_loss, 'b', 'Validation Loss')
plt.title('Loss Comparison')
plt.show()

## Missing Part

- Image Pre-processing process.
- Train operation